<a href="https://colab.research.google.com/github/kmalhotra18/Multi-API-Conversational-Chatbot/blob/main/Multiu_Modal_Image_Generation_and_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install anthropic

In [ ]:
# imports
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import json

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr # Import Gradio for UI

In [ ]:

# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()


In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages)
#     return response.choices[0].message.content

# gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("London")

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

**Lets go Multi-Modal!**

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

In [ ]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image           #Python Image Library

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",                       #Dall-e-3 model
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style", #prompt to display kind of image
            size="1024x1024",                       #size of image
            n=1,                                    #number of images to generate
            response_format="b64_json",              #format of response
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)          #Decode image to bytes
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("New York City")
display(image)

Let's make a function talker that uses **OpenAI's speech model** to generate Audio

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

In [ ]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

# def talker(message):
#     response = openai.audio.speech.create(
#         model="tts-1",
#         voice="alloy",                          #onxy is another sound
#         input=message)

#     audio_stream = BytesIO(response.content)
#     output_filename = "output_audio.mp3"
#     with open(output_filename, "wb") as f:
#         f.write(audio_stream.read())

#     # Play the generated audio
#     display(Audio(output_filename, autoplay=True))

# talker("Well, hi there")


def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=message
    )
    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    return output_filename  # Return path for Gradio



**Agent Framework**

In [ ]:
# def chat(history):                                                                              #usual chat function that takes message and history for OpenAi and calls response
#     messages = [{"role": "system", "content": system_message}] + history
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
#     image = None

#     if response.choices[0].finish_reason=="tool_calls":                                         #use of tools
#         message = response.choices[0].message
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         image = artist(city)                                                                    #image generation - if model runs tool to find price of ticket, then image of city will be generated
#         response = openai.chat.completions.create(model=MODEL, messages=messages)

#     reply = response.choices[0].message.content
#     history += [{"role":"assistant", "content":reply}]

#     # Comment out or delete the next line if you'd rather skip Audio for now..
#     talker(reply)                                                                              #audio generation - speak response

#     return history, image


def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]

    audio_path = talker(reply)  # generate and return audio

    return history, image, audio_path


In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

# with gr.Blocks() as ui:
#     with gr.Row():
#         chatbot = gr.Chatbot(height=500, type="messages")
#         image_output = gr.Image(height=500)
#     with gr.Row():
#         entry = gr.Textbox(label="Chat with our AI Assistant:")
#     with gr.Row():
#         clear = gr.Button("Clear")

#     def do_entry(message, history):
#         history += [{"role":"user", "content":message}]
#         return "", history

#     entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
#         chat, inputs=chatbot, outputs=[chatbot, image_output]
#     )
#     clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

# ui.launch(inbrowser=True)


with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
        audio_output = gr.Audio(label="AI Voice Response", autoplay=True)  # NEW!
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role": "user", "content": message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output, audio_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)